In [393]:
import sys
import numpy as np
import cv2 as cv
import igraph as ig
from GMM import GaussianMixture



DRAW_BG = {'val': 0}
DRAW_FG = {'val': 1}
DRAW_PR_BG = {'val': 2}
DRAW_PR_FG = {'val': 3}

# img = np.array(
# [
#     [[1,2,3], [4,5,6], [7,8,9], [10,11,12], [13,14,15] ],
#     [[16,17,18], [12,29,21], [21,23,24], [12,44,17], [32,23,22] ],
#     [[11,89,14], [23,41,11], [23,75,12], [13,47,27], [24,81,21] ],
#     [[21,79,24], [24,42,41], [23,65,92], [14,45,37], [44,83,12] ],
#     [[31,69,34], [12,43,31], [27,25,12], [51,55,57], [34,85,45] ],
#     [[41,19,44], [23,24,51], [28,55,32], [16,54,67], [64,30,33] ],
#     [[51,29,54], [33,25,61], [29,35,52], [31,31,77], [39,20,62] ],
#     [[61,49,64], [43,26,81], [45,25,62], [19,33,87], [32,80,82] ],
# ]
# )

img = np.array(
[
    [[159, 192, 201], [160, 193, 202], [162, 197, 207], [167, 203, 213], [166, 204, 216], [159, 198, 212], [158, 195, 199]],
    [[159, 192, 201], [156, 192, 200], [159, 194, 204], [163, 201, 213], [162, 201, 215], [152, 193, 208], [135, 177, 190]],
    [[156, 189, 198], [149, 185, 193], [150, 185, 195], [155, 193, 205], [157, 196, 210], [150, 191, 206], [142, 182, 187]],
    [[163, 197, 203], [152, 188, 196], [149, 184, 194], [153, 189, 199], [152, 192, 204], [146, 188, 201], [151, 189, 194]],
    [[150, 187, 191], [158, 195, 199], [160, 196, 202], [151, 189, 194], [145, 182, 190], [143, 182, 191], [145, 184, 198]],
    [[154, 191, 195], [158, 195, 199], [158, 195, 199], [150, 188, 192], [146, 184, 189], [150, 187, 195], [159, 194, 204]],
    [[151, 189, 194], [152, 190, 195], [149, 187, 192], [142, 182, 187], [142, 181, 189], [147, 186, 194], [132, 171, 186]],
    [[150, 187, 195], [146, 185, 193], [143, 182, 190], [141, 180, 188], [143, 182, 190], [147, 186, 195], [144, 185, 194]],
    [[150, 189, 198], [146, 187, 196], [144, 185, 194], [146, 187, 196], [148, 189, 198], [148, 188, 200], [152, 193, 208]],
    [[141, 180, 194], [140, 180, 192], [141, 180, 194], [145, 185, 197], [145, 184, 198], [142, 182, 194], [153, 189, 199]]
]
)

rows, cols, _= img.shape
comp_idxs = np.zeros((rows,cols), dtype=np.uint32)
mask = np.zeros(img.shape[:2], dtype=np.uint8)  # mask initialized to PR_BG
bgd_gmm = None
fgd_gmm = None

gamma = 50  # Best gamma suggested in paper formula (5)
beta = 0
left_V = np.empty((rows, cols - 1))
upleft_V = np.empty((rows - 1, cols - 1))
up_V = np.empty((rows - 1, cols))
upright_V = np.empty((rows - 1, cols - 1))



gc_graph = None
gc_graph_capacity = None           # Edge capacities
gc_source = cols * rows  # "object" terminal S
gc_sink = gc_source + 1       # "background" terminal T

print(img.shape)
print(mask, '\n')


(10, 7, 3)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] 



In [394]:
mask[2:7,1:5] = DRAW_PR_FG['val']
print(mask, '\n')

bgd_indexes = np.where(np.logical_or(
    mask == DRAW_BG['val'], mask == DRAW_PR_BG['val']))

fgd_indexes = np.where(np.logical_or(
    mask == DRAW_FG['val'], mask == DRAW_PR_FG['val']))


print('(pr_)bgd count: ', bgd_indexes, '\n \n(pr_)fgd count: ', fgd_indexes)
print('\n\n(pr_)bgd count: ', bgd_indexes[0].size, '(pr_)fgd count:', fgd_indexes[0].size, '\n')

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 3 3 3 3 0 0]
 [0 3 3 3 3 0 0]
 [0 3 3 3 3 0 0]
 [0 3 3 3 3 0 0]
 [0 3 3 3 3 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] 

(pr_)bgd count:  (array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4,
       4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9,
       9, 9, 9, 9, 9, 9], dtype=int64), array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 5, 6, 0, 5, 6, 0, 5,
       6, 0, 5, 6, 0, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6], dtype=int64)) 
 
(pr_)fgd count:  (array([2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
      dtype=int64), array([1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4],
      dtype=int64))


(pr_)bgd count:  50 (pr_)fgd count: 20 



In [395]:
# print("ini bgd_indexes : ", img[bgd_indexes])
# print("ini fgd_indexes : ", img[fgd_indexes], '\n')

'''Init GMM'''

bgd_gmm = GaussianMixture(img[bgd_indexes])
print('\n \n \n')
fgd_gmm = GaussianMixture(img[fgd_indexes])

ini covariance awal : [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
label dari fit yaitu  [4 2 2 0 0 3 4 2 2 4 2 3 3 0 2 4 1 3 0 1 1 1 1 4 2 0 1 3 0 0 0 2 0 3 0 0 4
 0 1 3 4 4 0 0 1 1 1 2 3 4]
bilangan unik =  (array([0, 1, 2, 3, 4]), array([14, 10,  9,  8,  9], dtype=int64))
n samples [14. 10.  9.  8.  9.]
ini coefs  [0.28 0.2  0.18 0.16 0.18]
ini fit dari mean [[148.         187.14285714 198.07142857]
 [145.2        184.3        193.2       ]
 [154.88888889 190.77777778 200.44444444]
 [151.75       190.75       201.5       ]
 [153.22222222 190.88888889 200.        ]]
ini fit dari cov [[20.69444444  9.40277778  2.25      ]
 [ 9.40277778  5.86111111  2.375     ]
 [ 2.25        2.375      11.        ]]

 
 

ini covariance awal : [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0

In [396]:
"""Step 1: Assign GMM"""

print('comp idxs untuk bgd')
bgd_gmm.which_component(img[bgd_indexes])
print('\ncomp idxs untuk fgd')
fgd_gmm.which_component(img[fgd_indexes])

comp idxs untuk bgd
target :  [[159 192 201]
 [160 193 202]
 [162 197 207]
 [167 203 213]
 [166 204 216]
 [159 198 212]
 [158 195 199]
 [159 192 201]
 [156 192 200]
 [159 194 204]
 [163 201 213]
 [162 201 215]
 [152 193 208]
 [135 177 190]
 [156 189 198]
 [150 191 206]
 [142 182 187]
 [163 197 203]
 [146 188 201]
 [151 189 194]
 [150 187 191]
 [143 182 191]
 [145 184 198]
 [154 191 195]
 [150 187 195]
 [159 194 204]
 [151 189 194]
 [147 186 194]
 [132 171 186]
 [150 187 195]
 [146 185 193]
 [143 182 190]
 [141 180 188]
 [143 182 190]
 [147 186 195]
 [144 185 194]
 [150 189 198]
 [146 187 196]
 [144 185 194]
 [146 187 196]
 [148 189 198]
 [148 188 200]
 [152 193 208]
 [141 180 194]
 [140 180 192]
 [141 180 194]
 [145 185 197]
 [145 184 198]
 [142 182 194]
 [153 189 199]]
hasil res
score =  [1.04776219e-04 1.14716191e-04 1.29160075e-03 1.46863875e-03
 1.28512632e-03 1.73357607e-03 2.66270575e-04 1.04776219e-04
 2.45082987e-03 1.32787213e-03 2.31693714e-03 1.00469714e-03
 1.30029727e-03 2

LinAlgError: Singular matrix

In [ ]:
fgd_gmm.fit(img[fgd_indexes],comp_idxs[fgd_indexes])
bgd_gmm.fit(img[bgd_indexes],comp_idxs[bgd_indexes])

label dari fit yaitu  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
bilangan unik =  (array([0], dtype=uint32), array([20], dtype=int64))
n samples [20.  0.  0.  0.  0.]
ini coefs  [1. 0. 0. 0. 0.]
ini fit dari mean [[151.4        188.8        196.15      ]
 [153.5        191.5        197.75      ]
 [151.4        188.2        197.        ]
 [151.28571429 188.57142857 195.71428571]
 [  0.           0.           0.        ]]
ini fit dari cov [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
label dari fit yaitu  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
bilangan unik =  (array([0], dtype=uint32), array([50], dtype=int64))
n samples [50.  0.  0.  0.  0.]
ini coefs  [1. 0. 0. 0. 0.]
ini fit dari mean [[150.22       188.48       198.42      ]
 [149.57142857 186.85714286 196.        ]
 [145.71428571 184.85714286 195.28571429]
 [156.5        194.25       204.375     ]
 [150.07142857 189.64285714 199.92857143]]
ini fit dari cov [[47.91758242 42.64285

In [ ]:
_left_diff = img[:, 1:] - img[:, :-1]
_upleft_diff = img[1:, 1:] - img[:-1, :-1]
_up_diff = img[1:, :] - img[:-1, :]
_upright_diff = img[1:, :-1] - img[:-1, 1:]

beta = np.sum(np.square(_left_diff)) + np.sum(np.square(_upleft_diff)) + \
np.sum(np.square(_up_diff)) + \
np.sum(np.square(_upright_diff))
beta = 1 / (2 * beta / (
# Each pixel has 4 neighbors (left, upleft, up, upright)
4 * cols * rows
# The 1st column doesn't have left, upleft and the last column doesn't have upright
- 3 * cols
- 3 * rows  # The first row doesn't have upleft, up and upright
+ 2))  # The first and last pixels in the 1st row are removed twice
# print('Beta:', beta)

# Smoothness term V described in formula (11)
left_V = gamma * np.exp(-beta * np.sum(
np.square(_left_diff), axis=2))
upleft_V = gamma / np.sqrt(2) * np.exp(-beta * np.sum(
np.square(_upleft_diff), axis=2))
up_V = gamma * np.exp(-beta * np.sum(
np.square(_up_diff), axis=2))
upright_V = gamma / np.sqrt(2) * np.exp(-beta * np.sum(
np.square(_upright_diff), axis=2))

In [400]:
bgd_indexes = np.where(mask.reshape(-1) == DRAW_BG['val'])
fgd_indexes = np.where(mask.reshape(-1) == DRAW_FG['val'])
pr_indexes = np.where(mask.reshape(-1) == DRAW_PR_FG['val'])


edges = []
gc_graph_capacity = []

print('edges awal :', len(edges), len(gc_graph_capacity))

# t-links
edges.extend(list(zip([gc_source] * pr_indexes[0].size, pr_indexes[0])))
_D = -np.log(bgd_gmm.calc_prob(img.reshape(-1, 3)[pr_indexes]))
# _D = bgd_gmm.calc_prob(img.reshape(-1, 3)[pr_indexes]
print('hasil _D source = ', _D.shape)
gc_graph_capacity.extend(_D.tolist())
# assert len(edges) == len(gc_graph_capacity)
print('panjang edge :', len(edges), len(gc_graph_capacity))
print('edges 1 :', edges, '\n', gc_graph_capacity), '\n'



# print(mask, '\n')
# print('TB count: %d, TF count: %d, TU count: %d' % (
#     len(bgd_indexes[0]), len(fgd_indexes[0]), len(pr_indexes[0])))

# print(bgd_indexes)
# print(pr_indexes)

edges awal : 0 0
score =  [0.00151154 0.00065103 0.00583263 0.00231674 0.00203132 0.00055873
 0.00271312 0.00584811 0.00026627 0.00100339 0.00071388 0.00205306
 0.00026627 0.00026627 0.00026906 0.0005628  0.00071627 0.00067845
 0.00031632 0.00337014]
score len =  20
score =  [4.66201507e-04 7.09044276e-06 8.52745133e-06 1.09821799e-09
 7.05551440e-04 4.36431218e-06 1.88499082e-04 7.31628185e-06
 1.29230299e-04 5.76271344e-06 1.43750727e-02 1.22265503e-03
 1.29230299e-04 1.29230299e-04 1.63683718e-02 1.05780146e-02
 9.11440590e-03 2.13231104e-02 3.98173612e-03 1.14257744e-02]
score len =  20
score =  [7.83027922e-03 5.78329556e-03 6.84830320e-03 1.09820567e-03
 1.01809046e-02 4.44401077e-03 1.43708162e-02 2.88088874e-03
 1.06309779e-05 5.73920549e-04 1.23916624e-04 4.34542981e-03
 1.06309779e-05 1.06309779e-05 2.40959136e-05 1.96322452e-04
 1.01869416e-04 1.65265961e-04 2.59235957e-05 1.82591868e-03]
score len =  20
score =  [5.43765862e-04 1.27371888e-06 7.74772516e-03 7.32076680e-03
 

(None, '\n')

In [ ]:
"""Step 3 in Figure 3: Estimate segmentation"""
mincut = gc_graph.st_mincut(
gc_source, gc_sink, gc_graph_capacity)
print('foreground pixels: %d, background pixels: %d' % (
len(mincut.partition[0]), len(mincut.partition[1])))
# print('jumlah partisi: ', mincut.partition)

pr_indexes = np.where(np.logical_or(
mask == DRAW_PR_BG['val'], mask == DRAW_PR_FG['val']))
img_indexes = np.arange(rows * cols,
                    dtype=np.uint32).reshape(rows, cols)
mask[pr_indexes] = np.where(np.isin(img_indexes[pr_indexes], mincut.partition[0]),
                                DRAW_PR_FG['val'], DRAW_PR_BG['val'])

                                

print(mask)

foreground pixels: 1, background pixels: 71
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 2 2 2 2 0 0]
 [0 2 2 2 2 0 0]
 [0 2 2 2 2 0 0]
 [0 2 2 2 2 0 0]
 [0 2 2 2 2 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
